- fix data quality issues
- transform json string to json 
- write to silver

In [0]:
select * from dea_course_catalog_gizmobox.bronze.v_orders v

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_fixed AS
select
  value,
  regexp_replace(
    value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"'
  ) AS fixed_value
from
  dea_course_catalog_gizmobox.bronze.v_orders

Trnasform json string to json object
- shcema_of_json : will mention schema of json
- from_json : will convert json string to json object , uses schema of json

In [0]:
select
  schema_of_json(fixed_value) as schema_of_json,
  from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_object,
  fixed_value
from
  tv_orders_fixed
LIMIT 13

In [0]:
DROP TABLE dea_course_catalog_gizmobox.silver.orders;
CREATE TABLE IF NOT EXISTS dea_course_catalog_gizmobox.silver.orders AS
select
  from_json(
    fixed_value,
    'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'
  ) as json_value
from
  tv_orders_fixed;

In [0]:
select * from dea_course_catalog_gizmobox.silver.orders

In [0]:
DESCRIBE DETAIL dea_course_catalog_gizmobox.silver.orders